In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
def read_page_get_links(keywords: str) -> "list":
    """
    Recebe a query como input e retorna os links disponiveis na primeira página do repositório da UFPE.
    
    Exemplo de query:
    
    "machine+learning"
    
    Os espaços devem ser representados por um "+"
    
    """
    response = requests.get("https://repositorio.ufpe.br/simple-search?query={}".format(keywords))
    soup = BeautifulSoup(response.text.strip(), 'html.parser')
    tags = soup.find_all(name='table')
    tag_table = tags[0]
    tags_a = tag_table.find_all(name='a')
    
    urls = []
    
    for i in range(0, len(tags_a), 2):
        link = tags_a[i]["href"]
        urls.append(link)
    
    return urls

In [3]:
def read_table_get_title_and_abstract(info_table: str) -> "dict":
    """
    Recebe uma tabela de informações HTML e extrai o title e abstract
    
    """
    all_titles = info_table.find_all(name='td', attrs={'class': 'dc_title'})
    title = all_titles[-1].contents[0]

    all_abst = info_table.find_all(name='td', attrs={'class': 'dc_description_abstract'})
    abst = all_abst[-1].contents[0]

    dict_thesis_info = {}
    dict_thesis_info = {
        'title': title,
        'abstract': abst,
    }
    return dict_thesis_info

In [4]:
urls = read_page_get_links("economia+brasileira")

In [5]:
info_table = []

for url in urls:
    response = requests.get(f'https://repositorio.ufpe.br/{url}', allow_redirects=True)
    soup = BeautifulSoup(response.text.strip(), 'html.parser')
    tags_table = soup.find_all(name='table', attrs={'class': 'itemDisplayTable'})
    info_table.append(tags_table[0])

In [6]:
counter = 1

for table in info_table:
    output = read_table_get_title_and_abstract(table)
    
    with open(f"./output/economiabrasileira/thesis{counter}.json", 'w') as json_file:
        json.dump(output, json_file)
        
    counter = counter + 1